# 04 — HTTP Agent (Webhook-style)

간단한 Flask 서버로 HTTP 엔드포인트(`/agent`)를 열어 **외부 시스템에서 POST 요청**으로 에이전트를 호출할 수 있게 만듭니다.

> 노트북에서는 로컬 실행 위주 예시를 제공합니다. 실제 배포는 별도 환경(예: Render, Azure App Service, GCP Cloud Run 등)을 사용하세요.


In [ ]:
%%bash
python - << 'PY'
import os, textwrap, pathlib
app_code = '''
from flask import Flask, request, jsonify
from openai import OpenAI
import os

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
app = Flask(__name__)

@app.route("/agent", methods=["POST"])
def agent():
    data = request.get_json(silent=True) or {}
    user = data.get("user", "")
    model = data.get("model", "gpt-4o-mini")
    messages = [
        {"role":"system","content":"You are a helpful HTTP agent."},
        {"role":"user","content": user}
    ]
    completion = client.chat.completions.create(model=model, messages=messages, max_tokens=200)
    return jsonify({"agent": completion.choices[0].message.content.strip()})
    
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=int(os.getenv("PORT", 5000)))
'''
pathlib.Path("app.py").write_text(textwrap.dedent(app_code), encoding="utf-8")
print("app.py created")
PY

## Run server (local)

> 이 셀을 실행하면 로컬에서 Flask 서버가 뜹니다. (노트북 환경에 따라 백그라운드 실행이 제한될 수 있습니다.)

```bash
python app.py
```

서버 실행 후 다른 터미널/노트북 셀에서 다음과 같이 호출해보세요:

```python
import requests
r = requests.post("http://127.0.0.1:5000/agent", json={"user":"간단한 일정 추천해줘"})
r.json()
```
